In [1]:
import pandas as pd
from gurobipy import *

In [2]:
with open('test_case.txt') as text_file:
    data = text_file.read().splitlines()

data

['Test Sequence with length 20:',
 'CGUCUUCACUACAGCAUCGG',
 '',
 'Test Sequence with length 30:',
 'GACCUUACUGGGUACGAUUUACUGGAGGAC',
 '',
 'Test Sequence with length 50:',
 'GGCCAGACUGGUGGUGUGACUCCAGGCUAACCGGAUACGCGUGCCUCGGG',
 '',
 'Test Sequence with length 100:',
 'UAUGCAGGUCGCGUUUUUCCACUGCCUAGAUAGCUCUGAGGGUACACUUAGUUCAGCACAUAAGAGGGAUCAUACUAGGUCCGCGUCUUACCUCCUACGA',
 '',
 'Test Sequence with length 200:',
 'CCAUGUGGACGAGUUCACGUUGCGGUCUGCCGCGAAGAAUGAGUCGUCCAAUUUUUUUUUCCUGUAUGACGGAUAGAGUGUAGGGUGGGUGAUAGUGUAACUCCGUCCCGGCGUUGUCUAUGGCAGUCUAAUUGUAGUGAGCCAAAAUAUUUCAGAGAGUAACCAUUACAGGCUACCACACGACCCUGAUCCUAUUGUAU',
 '']

In [3]:
s = data[7]
len(s)

50

In [4]:
m = Model("Position_Quartet")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-12


In [5]:
m.setParam('OutputFlag',True)
m.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


# Pre-Processing Data

In [6]:
ONE_N = list(range(1,len(s)+1))
ONE_N

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

## A_i, U_i, C_i, G_i

In [7]:
a = {}
u = {}
c = {}
g = {}

In [8]:
index = 1
for n in s:
    if (n == 'A'):
        a[index] = 1
        u[index] = 0
        c[index] = 0
        g[index] = 0
    if (n == 'U'):
        a[index] = 0
        u[index] = 1
        c[index] = 0
        g[index] = 0
    if (n == 'C'):
        a[index] = 0
        u[index] = 0
        c[index] = 1
        g[index] = 0
    if (n == 'G'):
        a[index] = 0
        u[index] = 0
        c[index] = 0
        g[index] = 1
    
    index += 1

In [9]:
print(a)
print(u)
print(c)
print(g)
print(s)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0}
{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 1, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 1, 43: 0, 44: 0, 45: 0, 46: 1, 47: 0, 48: 0, 49: 0, 50: 0}
{1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 0, 22: 1, 23: 1, 24: 0, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 1, 41: 0, 42: 0, 43: 0, 44: 1, 45: 1, 46: 0, 47

## X_ij

In [10]:
ls_tup = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        ls_tup.append((i,j))
        
x = m.addVars(ls_tup,name='X_ij',vtype=GRB.BINARY)
x

{(1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,


## d_ij Minimum Distance Constraint

In [11]:
ls_dtup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_dtup.append((i,j))

d = {}
d

{}

In [12]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        w = min(j-i,len(s)-j+i)
        d[i,j] = w

d

{(1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (1, 5): 4,
 (1, 6): 5,
 (1, 7): 6,
 (1, 8): 7,
 (1, 9): 8,
 (1, 10): 9,
 (1, 11): 10,
 (1, 12): 11,
 (1, 13): 12,
 (1, 14): 13,
 (1, 15): 14,
 (1, 16): 15,
 (1, 17): 16,
 (1, 18): 17,
 (1, 19): 18,
 (1, 20): 19,
 (1, 21): 20,
 (1, 22): 21,
 (1, 23): 22,
 (1, 24): 23,
 (1, 25): 24,
 (1, 26): 25,
 (1, 27): 24,
 (1, 28): 23,
 (1, 29): 22,
 (1, 30): 21,
 (1, 31): 20,
 (1, 32): 19,
 (1, 33): 18,
 (1, 34): 17,
 (1, 35): 16,
 (1, 36): 15,
 (1, 37): 14,
 (1, 38): 13,
 (1, 39): 12,
 (1, 40): 11,
 (1, 41): 10,
 (1, 42): 9,
 (1, 43): 8,
 (1, 44): 7,
 (1, 45): 6,
 (1, 46): 5,
 (1, 47): 4,
 (1, 48): 3,
 (1, 49): 2,
 (1, 50): 1,
 (2, 3): 1,
 (2, 4): 2,
 (2, 5): 3,
 (2, 6): 4,
 (2, 7): 5,
 (2, 8): 6,
 (2, 9): 7,
 (2, 10): 8,
 (2, 11): 9,
 (2, 12): 10,
 (2, 13): 11,
 (2, 14): 12,
 (2, 15): 13,
 (2, 16): 14,
 (2, 17): 15,
 (2, 18): 16,
 (2, 19): 17,
 (2, 20): 18,
 (2, 21): 19,
 (2, 22): 20,
 (2, 23): 21,
 (2, 24): 22,
 (2, 25): 23,
 (2, 26): 24,
 (2, 27): 25,
 (2, 2

In [13]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if d[i,j] < 3:
            x[i,j] = 0

x

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## f_ij

In [14]:
ls_ftup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_ftup.append((i,j))
        


        
f = {}
f

{}

In [15]:
temp = ''
temp = 'x' + s
temp
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if (((temp[i] == 'C') & (temp[j] == 'G')) or ((temp[i] == 'G') & (temp[j] == 'C'))):
            f[i,j] = 3
        elif (((temp[i] == 'A') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'A'))):
            f[i,j] = 2
        elif (((temp[i] == 'G') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'G'))):
            f[i,j] = 0.1
        elif (((temp[i] == 'A') & (temp[j] == 'C')) or ((temp[i] == 'C') & (temp[j] == 'A'))):
            f[i,j] = 0.05
        else:
            f[i,j] = 0
            
f

{(1, 2): 0,
 (1, 3): 3,
 (1, 4): 3,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 0,
 (1, 8): 3,
 (1, 9): 0.1,
 (1, 10): 0,
 (1, 11): 0,
 (1, 12): 0.1,
 (1, 13): 0,
 (1, 14): 0,
 (1, 15): 0.1,
 (1, 16): 0,
 (1, 17): 0.1,
 (1, 18): 0,
 (1, 19): 0,
 (1, 20): 3,
 (1, 21): 0.1,
 (1, 22): 3,
 (1, 23): 3,
 (1, 24): 0,
 (1, 25): 0,
 (1, 26): 0,
 (1, 27): 3,
 (1, 28): 0.1,
 (1, 29): 0,
 (1, 30): 0,
 (1, 31): 3,
 (1, 32): 3,
 (1, 33): 0,
 (1, 34): 0,
 (1, 35): 0,
 (1, 36): 0.1,
 (1, 37): 0,
 (1, 38): 3,
 (1, 39): 0,
 (1, 40): 3,
 (1, 41): 0,
 (1, 42): 0.1,
 (1, 43): 0,
 (1, 44): 3,
 (1, 45): 3,
 (1, 46): 0.1,
 (1, 47): 3,
 (1, 48): 0,
 (1, 49): 0,
 (1, 50): 0,
 (2, 3): 3,
 (2, 4): 3,
 (2, 5): 0,
 (2, 6): 0,
 (2, 7): 0,
 (2, 8): 3,
 (2, 9): 0.1,
 (2, 10): 0,
 (2, 11): 0,
 (2, 12): 0.1,
 (2, 13): 0,
 (2, 14): 0,
 (2, 15): 0.1,
 (2, 16): 0,
 (2, 17): 0.1,
 (2, 18): 0,
 (2, 19): 0,
 (2, 20): 3,
 (2, 21): 0.1,
 (2, 22): 3,
 (2, 23): 3,
 (2, 24): 0,
 (2, 25): 0,
 (2, 26): 0,
 (2, 27): 3,
 (2, 28): 0.1,
 (2, 29): 

## Q_ij

In [16]:
ls_qtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_qtup.append((i,j))

q_ij = m.addVars(ls_qtup,name='Q_ij',vtype=GRB.BINARY)
q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_nj

In [17]:
ls_njtup = []
for j in range(3,len(s)):
    ls_njtup.append((len(s),j))
    
    
q_nj = m.addVars(ls_njtup,name='Q_nj',vtype=GRB.BINARY)
q_nj

{(50, 3): <gurobi.Var *Awaiting Model Update*>,
 (50, 4): <gurobi.Var *Awaiting Model Update*>,
 (50, 5): <gurobi.Var *Awaiting Model Update*>,
 (50, 6): <gurobi.Var *Awaiting Model Update*>,
 (50, 7): <gurobi.Var *Awaiting Model Update*>,
 (50, 8): <gurobi.Var *Awaiting Model Update*>,
 (50, 9): <gurobi.Var *Awaiting Model Update*>,
 (50, 10): <gurobi.Var *Awaiting Model Update*>,
 (50, 11): <gurobi.Var *Awaiting Model Update*>,
 (50, 12): <gurobi.Var *Awaiting Model Update*>,
 (50, 13): <gurobi.Var *Awaiting Model Update*>,
 (50, 14): <gurobi.Var *Awaiting Model Update*>,
 (50, 15): <gurobi.Var *Awaiting Model Update*>,
 (50, 16): <gurobi.Var *Awaiting Model Update*>,
 (50, 17): <gurobi.Var *Awaiting Model Update*>,
 (50, 18): <gurobi.Var *Awaiting Model Update*>,
 (50, 19): <gurobi.Var *Awaiting Model Update*>,
 (50, 20): <gurobi.Var *Awaiting Model Update*>,
 (50, 21): <gurobi.Var *Awaiting Model Update*>,
 (50, 22): <gurobi.Var *Awaiting Model Update*>,
 (50, 23): <gurobi.Var *Awa

## w_ij

In [18]:
ls_wtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_wtup.append((i,j))
        
w_ij = {}
w_ij

{}

In [19]:
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        if ((a[i] == 1) & (u[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 9
        elif ((a[i] == 1) & (u[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((a[i] == 1) & (u[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 24
        elif ((a[i] == 1) & (u[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 13
            
        elif ((c[i] == 1) & (g[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 22
        elif ((c[i] == 1) & (g[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 33
        elif ((c[i] == 1) & (g[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 34
        elif ((c[i] == 1) & (g[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 24
            
        elif ((g[i] == 1) & (c[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((g[i] == 1 )& (c[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 24
        elif ((g[i] == 1) & (c[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 33
        elif ((g[i] == 1) & (c[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 21
            
        elif ((u[i] == 1 )& (a[j] == 1) & (a[i+1] == 1) & (u[j-1] == 1)):
            w_ij[i,j] = 12
        elif ((u[i] == 1) & (a[j] == 1) & (c[i+1] == 1) & (g[j-1] == 1)):
            w_ij[i,j] = 23
        elif ((u[i] == 1) & (a[j] == 1) & (g[i+1] == 1) & (c[j-1] == 1)):
            w_ij[i,j] = 21
        elif ((u[i] == 1) & (a[j] == 1) & (u[i+1] == 1) & (a[j-1] == 1)):
            w_ij[i,j] = 16
        else:
            w_ij[i,j] = 1
            

w_ij

{(1, 4): 33,
 (1, 5): 1,
 (1, 6): 1,
 (1, 7): 1,
 (1, 8): 1,
 (1, 9): 1,
 (1, 10): 1,
 (1, 11): 1,
 (1, 12): 1,
 (1, 13): 1,
 (1, 14): 1,
 (1, 15): 1,
 (1, 16): 1,
 (1, 17): 1,
 (1, 18): 1,
 (1, 19): 1,
 (1, 20): 1,
 (1, 21): 1,
 (1, 22): 1,
 (1, 23): 33,
 (1, 24): 1,
 (1, 25): 1,
 (1, 26): 1,
 (1, 27): 1,
 (1, 28): 1,
 (1, 29): 1,
 (1, 30): 1,
 (1, 31): 1,
 (1, 32): 33,
 (1, 33): 1,
 (1, 34): 1,
 (1, 35): 1,
 (1, 36): 1,
 (1, 37): 1,
 (1, 38): 1,
 (1, 39): 1,
 (1, 40): 1,
 (1, 41): 1,
 (1, 42): 1,
 (1, 43): 1,
 (1, 44): 1,
 (1, 45): 33,
 (1, 46): 1,
 (1, 47): 1,
 (1, 48): 1,
 (1, 49): 1,
 (1, 50): 1,
 (2, 5): 1,
 (2, 6): 1,
 (2, 7): 1,
 (2, 8): 1,
 (2, 9): 1,
 (2, 10): 1,
 (2, 11): 1,
 (2, 12): 1,
 (2, 13): 1,
 (2, 14): 1,
 (2, 15): 1,
 (2, 16): 1,
 (2, 17): 1,
 (2, 18): 1,
 (2, 19): 1,
 (2, 20): 1,
 (2, 21): 1,
 (2, 22): 1,
 (2, 23): 1,
 (2, 24): 1,
 (2, 25): 1,
 (2, 26): 1,
 (2, 27): 24,
 (2, 28): 1,
 (2, 29): 1,
 (2, 30): 1,
 (2, 31): 1,
 (2, 32): 1,
 (2, 33): 1,
 (2, 34): 1,
 (2, 

## w_nj

In [20]:
ls_wnjtup = []
for j in range(3,len(s)):
    ls_wnjtup.append((len(s),j))


w_nj = {}
w_nj

{}

In [21]:
for j in range(3,len(s)):
    if ((a[len(s)] == 1) & (u[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 9
    elif ((a[len(s)] == 1) & (u[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((a[len(s)] == 1) & (u[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 24
    elif ((a[len(s)] == 1) & (u[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 13

    elif ((c[len(s)] == 1) & (g[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 22
    elif ((c[len(s)] == 1) & (g[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 33
    elif ((c[len(s)] == 1) & (g[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 34
    elif ((c[len(s)] == 1) & (g[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 24

    elif ((g[len(s)] == 1) & (c[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((g[len(s)] == 1 )& (c[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 24
    elif ((g[len(s)] == 1) & (c[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 33
    elif ((g[len(s)] == 1) & (c[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 21

    elif ((u[len(s)] == 1 )& (a[j] == 1) & (a[1] == 1) & (u[j-1] == 1)):
        w_nj[len(s),j] = 12
    elif ((u[len(s)] == 1) & (a[j] == 1) & (c[1] == 1) & (g[j-1] == 1)):
        w_nj[len(s),j] = 23
    elif ((u[len(s)] == 1) & (a[j] == 1) & (g[1] == 1) & (c[j-1] == 1)):
        w_nj[len(s),j] = 21
    elif ((u[len(s)] == 1) & (a[j] == 1) & (u[1] == 1) & (a[j-1] == 1)):
        w_nj[len(s),j] = 16
    else:
        w_nj[len(s),j] = 1


w_nj

{(50, 3): 1,
 (50, 4): 33,
 (50, 5): 1,
 (50, 6): 1,
 (50, 7): 1,
 (50, 8): 1,
 (50, 9): 1,
 (50, 10): 1,
 (50, 11): 1,
 (50, 12): 1,
 (50, 13): 1,
 (50, 14): 1,
 (50, 15): 1,
 (50, 16): 1,
 (50, 17): 1,
 (50, 18): 1,
 (50, 19): 1,
 (50, 20): 1,
 (50, 21): 1,
 (50, 22): 1,
 (50, 23): 33,
 (50, 24): 1,
 (50, 25): 1,
 (50, 26): 1,
 (50, 27): 1,
 (50, 28): 1,
 (50, 29): 1,
 (50, 30): 1,
 (50, 31): 1,
 (50, 32): 33,
 (50, 33): 1,
 (50, 34): 1,
 (50, 35): 1,
 (50, 36): 1,
 (50, 37): 1,
 (50, 38): 1,
 (50, 39): 1,
 (50, 40): 1,
 (50, 41): 1,
 (50, 42): 1,
 (50, 43): 1,
 (50, 44): 1,
 (50, 45): 33,
 (50, 46): 1,
 (50, 47): 1,
 (50, 48): 1,
 (50, 49): 1}

## P_ij

In [22]:
ls_pijtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_pijtup.append((i,j))
        
p_ij = m.addVars(ls_pijtup,name='P_ij',vtype=GRB.BINARY)
p_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## P_nj

In [23]:
ls_pnjtup = []
for j in range(3,len(s)):
    ls_pnjtup.append((len(s),j))


p_nj = m.addVars(ls_pnjtup,name='P_nj',vtype=GRB.BINARY)
p_nj

{(50, 3): <gurobi.Var *Awaiting Model Update*>,
 (50, 4): <gurobi.Var *Awaiting Model Update*>,
 (50, 5): <gurobi.Var *Awaiting Model Update*>,
 (50, 6): <gurobi.Var *Awaiting Model Update*>,
 (50, 7): <gurobi.Var *Awaiting Model Update*>,
 (50, 8): <gurobi.Var *Awaiting Model Update*>,
 (50, 9): <gurobi.Var *Awaiting Model Update*>,
 (50, 10): <gurobi.Var *Awaiting Model Update*>,
 (50, 11): <gurobi.Var *Awaiting Model Update*>,
 (50, 12): <gurobi.Var *Awaiting Model Update*>,
 (50, 13): <gurobi.Var *Awaiting Model Update*>,
 (50, 14): <gurobi.Var *Awaiting Model Update*>,
 (50, 15): <gurobi.Var *Awaiting Model Update*>,
 (50, 16): <gurobi.Var *Awaiting Model Update*>,
 (50, 17): <gurobi.Var *Awaiting Model Update*>,
 (50, 18): <gurobi.Var *Awaiting Model Update*>,
 (50, 19): <gurobi.Var *Awaiting Model Update*>,
 (50, 20): <gurobi.Var *Awaiting Model Update*>,
 (50, 21): <gurobi.Var *Awaiting Model Update*>,
 (50, 22): <gurobi.Var *Awaiting Model Update*>,
 (50, 23): <gurobi.Var *Awa

## L_ij

In [24]:
ls_lijtup = []
for i in range(1,len(s)-2):
    for j in range(i+3,len(s)+1):
        ls_lijtup.append((i,j))
        
l_ij = m.addVars(ls_lijtup,name='L_ij',vtype=GRB.BINARY)
l_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## L_nj

In [25]:
ls_lnjtup = []
for j in range(3,len(s)):
    ls_lnjtup.append((len(s),j))


l_nj = m.addVars(ls_lnjtup,name='L_nj',vtype=GRB.BINARY)
l_nj

{(50, 3): <gurobi.Var *Awaiting Model Update*>,
 (50, 4): <gurobi.Var *Awaiting Model Update*>,
 (50, 5): <gurobi.Var *Awaiting Model Update*>,
 (50, 6): <gurobi.Var *Awaiting Model Update*>,
 (50, 7): <gurobi.Var *Awaiting Model Update*>,
 (50, 8): <gurobi.Var *Awaiting Model Update*>,
 (50, 9): <gurobi.Var *Awaiting Model Update*>,
 (50, 10): <gurobi.Var *Awaiting Model Update*>,
 (50, 11): <gurobi.Var *Awaiting Model Update*>,
 (50, 12): <gurobi.Var *Awaiting Model Update*>,
 (50, 13): <gurobi.Var *Awaiting Model Update*>,
 (50, 14): <gurobi.Var *Awaiting Model Update*>,
 (50, 15): <gurobi.Var *Awaiting Model Update*>,
 (50, 16): <gurobi.Var *Awaiting Model Update*>,
 (50, 17): <gurobi.Var *Awaiting Model Update*>,
 (50, 18): <gurobi.Var *Awaiting Model Update*>,
 (50, 19): <gurobi.Var *Awaiting Model Update*>,
 (50, 20): <gurobi.Var *Awaiting Model Update*>,
 (50, 21): <gurobi.Var *Awaiting Model Update*>,
 (50, 22): <gurobi.Var *Awaiting Model Update*>,
 (50, 23): <gurobi.Var *Awa

## Additional Constraints for Q_ij and Q_nj variables

In [26]:
m.addConstrs(2*q_ij[i,j] <= x[i,j] + x[i+1,j-1] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [27]:
m.addConstrs(2*q_nj[len(s),j] <= x[j,len(s)] + x[1,j-1] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

## Q_i,i+1

In [28]:
ls_ii1tup = []
for i in range(1,len(s)):
    ls_ii1tup.append((i,i+1))

m.addVars(ls_ii1tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)):
    q_ij[i,i+1] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_i,i+2

In [29]:
ls_ii2tup = []
for i in range(1,len(s)-1):
    ls_ii2tup.append((i,i+2))

m.addVars(ls_ii2tup,name='Q_ij',vtype=GRB.BINARY)
for i in range(1,len(s)-1):
    q_ij[i,i+2] = 0

q_ij

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (1, 21): <gurobi.Var *Awaiting Model Update*>,
 (1, 22): <gurobi.Var *Awaiting Model Update*>,
 (1, 23): <gurobi.Var *Awaiting Model Update*>,
 (1, 24): <gurobi.Var *Awaiting Model Update*>

## Q_n,n-1

In [30]:
q_nj[len(s),len(s)-1] = 0

## Q_n,n-2

In [31]:
q_nj[len(s),len(s)-2] = 0

# Additional Constraints for P_ij, P_nj, L_ij, L_nj

In [32]:
m.addConstrs(q_ij[i,j] >= p_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [33]:
m.addConstrs(q_ij[i,j] + (1-q_ij[i-1,j+1]) >= 2*p_ij[i,j] for i in range(2,len(s)-2) for j in range(i+3,len(s)))

{(2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (2, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 21): <gurobi.Constr *Awaiting Model Update*>,
 (2, 22): <gurobi.Constr *Awaiting Model Update*>,
 (2, 23): <gurobi.Constr *Awaiting Model Update*>,
 (2, 24): <gurobi.Constr *Awaiting M

In [34]:
m.addConstrs(q_ij[i,len(s)] + (1-q_ij[1,i-1]) >= 2*p_ij[i,len(s)] for i in range(3,len(s)-2))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

In [35]:
m.addConstrs(q_ij[1,j] + (1-q_nj[len(s),j+1]) >= 2 * p_ij[1,j] for j in range(4,len(s)-1))

{4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Update*>,
 25: <gurobi.Constr *Awaiting Model Upda

In [36]:
m.addConstrs(q_nj[len(s),j] >= p_nj[len(s),j] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

In [37]:
m.addConstrs(q_nj[len(s),j] + (1-q_ij[j+1,len(s)-1]) >= 2*p_nj[len(s),j] for j in range(3,len(s)-2))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

In [38]:
m.addConstrs(q_ij[i,j] >= l_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [39]:
m.addConstrs(q_ij[i,j] + (1-q_ij[i+1,j-1]) >= 2*l_ij[i,j] for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [40]:
m.addConstrs(q_nj[len(s),j] + (1-q_ij[1,j-1]) >= 2*l_nj[len(s),j] for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

In [41]:
m.addConstrs(p_ij[i,j] + l_ij[i,j] <= 1 for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))

{(1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Model Update*>,
 (1, 22): <gurobi.Constr *Awaiting Model Update*>,
 (1, 23): <gurobi.Constr *Awaiting Mo

In [42]:
m.addConstrs(p_nj[len(s),j] + l_nj[len(s),j] <= 1 for j in range(3,len(s)))

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Updat

## Non-Crossing Constraint

In [43]:
m.addConstrs(quicksum(x[k,l] for k in range(i+1,j) for l in [i for i in list(range(1,len(s)+1)) if i >= j]) <= (10**6)*(1-x[i,j]) for i in range(1,len(s)+1) for j in range(i+1,len(s)+1))

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Mode

## Character in At most one pair

In [44]:
m.addConstrs(quicksum([x[i,k] for i in range(1,k)]) + quicksum([x[k,j] for j in list(range(k+1,len(s)+1))]) <= 1 for k in ONE_N)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

# Objective

In [45]:
m.setObjective((quicksum(f[i,j]*x[i,j] for i in ONE_N for j in range(i+1,len(s)+1)) + 
               (quicksum((w_ij[i,j] * (q_ij[i,j] + p_ij[i,j] + l_ij[i,j])) for i in range(1,len(s)-2) for j in range(i+3,len(s)+1))) + 
               (quicksum((w_nj[len(s),j] * (q_nj[len(s),j] + p_nj[len(s),j] + l_nj[len(s),j])) for j in range(3,len(s))))),GRB.MAXIMIZE)

In [46]:
# Optimize the model
m.optimize()
# Print the result
status_code = {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED'}
status = m.status
print("The optimization status is {}".format(status_code[status]))
if status == 2:
# Retrieve variables value
    print("Optimal solution:")
    for v in m.getVars():
        if (v.x == 1):
            print("%s = %g" % (v.varName, v.x))
    print("Optimal objective value:\n{}".format(m.objVal))
    print("Runtime is:", m.Runtime)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8273 rows, 4847 columns and 267106 nonzeros
Model fingerprint: 0x4bf34fb6
Variable types: 0 continuous, 4847 integer (4847 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e-02, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 17.5500000
Sparsify removed 203454 nonzeros (83%)
Presolve removed 3159 rows and 159 columns
Presolve time: 4.76s
Presolved: 5114 rows, 4688 columns, 41936 nonzeros
Variable types: 48 continuous, 4640 integer (4164 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     349    7.0675750e+03   1.906219e+03   0.000000e+00      5s
    4013    8.2539472e+02   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 8.253947e+02, 4013 iterations, 0.43 seconds (0.07 

X_ij[23,49] = 0
X_ij[23,50] = 0
X_ij[24,25] = 0
X_ij[24,26] = 0
X_ij[24,27] = -0
X_ij[24,28] = -0
X_ij[24,29] = -0
X_ij[24,30] = -0
X_ij[24,31] = -0
X_ij[24,32] = -0
X_ij[24,33] = -0
X_ij[24,34] = -0
X_ij[24,35] = -0
X_ij[24,36] = -0
X_ij[24,37] = -0
X_ij[24,38] = -0
X_ij[24,39] = -0
X_ij[24,40] = -0
X_ij[24,41] = -0
X_ij[24,42] = -0
X_ij[24,43] = -0
X_ij[24,44] = -0
X_ij[24,45] = -0
X_ij[24,46] = 0
X_ij[24,47] = -0
X_ij[24,48] = -0
X_ij[24,49] = -0
X_ij[24,50] = -0
X_ij[25,26] = 0
X_ij[25,27] = 0
X_ij[25,28] = -0
X_ij[25,29] = -0
X_ij[25,30] = -0
X_ij[25,31] = -0
X_ij[25,32] = -0
X_ij[25,33] = -0
X_ij[25,34] = -0
X_ij[25,35] = -0
X_ij[25,36] = -0
X_ij[25,37] = -0
X_ij[25,38] = -0
X_ij[25,39] = -0
X_ij[25,40] = -0
X_ij[25,41] = -0
X_ij[25,42] = -0
X_ij[25,43] = -0
X_ij[25,44] = -0
X_ij[25,45] = 1
X_ij[25,46] = -0
X_ij[25,47] = -0
X_ij[25,48] = -0
X_ij[25,49] = -0
X_ij[25,50] = -0
X_ij[26,27] = 0
X_ij[26,28] = 0
X_ij[26,29] = -0
X_ij[26,30] = -0
X_ij[26,31] = -0
X_ij[26,32] = -0
X_ij[26

Q_ij[4,15] = -0
Q_ij[4,16] = 0
Q_ij[4,17] = -0
Q_ij[4,18] = 0
Q_ij[4,19] = -0
Q_ij[4,20] = -0
Q_ij[4,21] = -0
Q_ij[4,22] = -0
Q_ij[4,23] = -0
Q_ij[4,24] = -0
Q_ij[4,25] = -0
Q_ij[4,26] = -0
Q_ij[4,27] = -0
Q_ij[4,28] = -0
Q_ij[4,29] = -0
Q_ij[4,30] = -0
Q_ij[4,31] = -0
Q_ij[4,32] = -0
Q_ij[4,33] = -0
Q_ij[4,34] = -0
Q_ij[4,35] = -0
Q_ij[4,36] = -0
Q_ij[4,37] = -0
Q_ij[4,38] = -0
Q_ij[4,39] = -0
Q_ij[4,40] = -0
Q_ij[4,41] = -0
Q_ij[4,42] = -0
Q_ij[4,43] = -0
Q_ij[4,44] = -0
Q_ij[4,45] = -0
Q_ij[4,46] = -0
Q_ij[4,47] = -0
Q_ij[4,48] = -0
Q_ij[4,49] = 1
Q_ij[4,50] = -0
Q_ij[5,8] = 0
Q_ij[5,9] = 0
Q_ij[5,10] = -0
Q_ij[5,11] = -0
Q_ij[5,12] = -0
Q_ij[5,13] = -0
Q_ij[5,14] = -0
Q_ij[5,15] = -0
Q_ij[5,16] = -0
Q_ij[5,17] = -0
Q_ij[5,18] = -0
Q_ij[5,19] = -0
Q_ij[5,20] = -0
Q_ij[5,21] = 0
Q_ij[5,22] = -0
Q_ij[5,23] = -0
Q_ij[5,24] = -0
Q_ij[5,25] = -0
Q_ij[5,26] = -0
Q_ij[5,27] = -0
Q_ij[5,28] = 0
Q_ij[5,29] = -0
Q_ij[5,30] = -0
Q_ij[5,31] = -0
Q_ij[5,32] = -0
Q_ij[5,33] = -0
Q_ij[5,34] = -0
Q

Q_ij[40,47] = -0
Q_ij[40,48] = 0
Q_ij[40,49] = -0
Q_ij[40,50] = -0
Q_ij[41,44] = 0
Q_ij[41,45] = 0
Q_ij[41,46] = -0
Q_ij[41,47] = -0
Q_ij[41,48] = -0
Q_ij[41,49] = -0
Q_ij[41,50] = -0
Q_ij[42,45] = 0
Q_ij[42,46] = 0
Q_ij[42,47] = -0
Q_ij[42,48] = -0
Q_ij[42,49] = -0
Q_ij[42,50] = -0
Q_ij[43,46] = 0
Q_ij[43,47] = 0
Q_ij[43,48] = -0
Q_ij[43,49] = -0
Q_ij[43,50] = -0
Q_ij[44,47] = 0
Q_ij[44,48] = 0
Q_ij[44,49] = 0
Q_ij[44,50] = -0
Q_ij[45,48] = 0
Q_ij[45,49] = 0
Q_ij[45,50] = -0
Q_ij[46,49] = 0
Q_ij[46,50] = 0
Q_ij[47,50] = 0
Q_nj[50,3] = 0
Q_nj[50,4] = 0
Q_nj[50,5] = -0
Q_nj[50,6] = -0
Q_nj[50,7] = -0
Q_nj[50,8] = -0
Q_nj[50,9] = -0
Q_nj[50,10] = -0
Q_nj[50,11] = -0
Q_nj[50,12] = -0
Q_nj[50,13] = -0
Q_nj[50,14] = -0
Q_nj[50,15] = -0
Q_nj[50,16] = -0
Q_nj[50,17] = -0
Q_nj[50,18] = -0
Q_nj[50,19] = -0
Q_nj[50,20] = -0
Q_nj[50,21] = -0
Q_nj[50,22] = -0
Q_nj[50,23] = 0
Q_nj[50,24] = -0
Q_nj[50,25] = -0
Q_nj[50,26] = -0
Q_nj[50,27] = -0
Q_nj[50,28] = -0
Q_nj[50,29] = 0
Q_nj[50,30] = -0
Q_nj[5

P_ij[12,23] = -0
P_ij[12,24] = 0
P_ij[12,25] = -0
P_ij[12,26] = -0
P_ij[12,27] = -0
P_ij[12,28] = -0
P_ij[12,29] = -0
P_ij[12,30] = -0
P_ij[12,31] = -0
P_ij[12,32] = -0
P_ij[12,33] = -0
P_ij[12,34] = -0
P_ij[12,35] = -0
P_ij[12,36] = -0
P_ij[12,37] = -0
P_ij[12,38] = -0
P_ij[12,39] = -0
P_ij[12,40] = -0
P_ij[12,41] = -0
P_ij[12,42] = -0
P_ij[12,43] = -0
P_ij[12,44] = -0
P_ij[12,45] = -0
P_ij[12,46] = -0
P_ij[12,47] = -0
P_ij[12,48] = -0
P_ij[12,49] = -0
P_ij[12,50] = -0
P_ij[13,16] = 0
P_ij[13,17] = 0
P_ij[13,18] = 0
P_ij[13,19] = 0
P_ij[13,20] = -0
P_ij[13,21] = 1
P_ij[13,22] = -0
P_ij[13,23] = -0
P_ij[13,24] = -0
P_ij[13,25] = -0
P_ij[13,26] = -0
P_ij[13,27] = -0
P_ij[13,28] = -0
P_ij[13,29] = -0
P_ij[13,30] = -0
P_ij[13,31] = -0
P_ij[13,32] = 0
P_ij[13,33] = -0
P_ij[13,34] = -0
P_ij[13,35] = -0
P_ij[13,36] = -0
P_ij[13,37] = -0
P_ij[13,38] = -0
P_ij[13,39] = -0
P_ij[13,40] = -0
P_ij[13,41] = -0
P_ij[13,42] = -0
P_ij[13,43] = -0
P_ij[13,44] = -0
P_ij[13,45] = 0
P_ij[13,46] = -0
P_ij[

L_ij[5,37] = -0
L_ij[5,38] = -0
L_ij[5,39] = -0
L_ij[5,40] = -0
L_ij[5,41] = -0
L_ij[5,42] = -0
L_ij[5,43] = -0
L_ij[5,44] = -0
L_ij[5,45] = -0
L_ij[5,46] = 0
L_ij[5,47] = -0
L_ij[5,48] = -0
L_ij[5,49] = -0
L_ij[5,50] = 0
L_ij[6,9] = 0
L_ij[6,10] = 0
L_ij[6,11] = 0
L_ij[6,12] = 0
L_ij[6,13] = -0
L_ij[6,14] = -0
L_ij[6,15] = -0
L_ij[6,16] = -0
L_ij[6,17] = -0
L_ij[6,18] = -0
L_ij[6,19] = -0
L_ij[6,20] = -0
L_ij[6,21] = -0
L_ij[6,22] = -0
L_ij[6,23] = -0
L_ij[6,24] = -0
L_ij[6,25] = -0
L_ij[6,26] = -0
L_ij[6,27] = -0
L_ij[6,28] = -0
L_ij[6,29] = -0
L_ij[6,30] = -0
L_ij[6,31] = -0
L_ij[6,32] = -0
L_ij[6,33] = -0
L_ij[6,34] = -0
L_ij[6,35] = -0
L_ij[6,36] = -0
L_ij[6,37] = -0
L_ij[6,38] = -0
L_ij[6,39] = -0
L_ij[6,40] = -0
L_ij[6,41] = -0
L_ij[6,42] = -0
L_ij[6,43] = -0
L_ij[6,44] = -0
L_ij[6,45] = -0
L_ij[6,46] = -0
L_ij[6,47] = 0
L_ij[6,48] = -0
L_ij[6,49] = -0
L_ij[6,50] = 0
L_ij[7,10] = 0
L_ij[7,11] = 0
L_ij[7,12] = 0
L_ij[7,13] = 0
L_ij[7,14] = -0
L_ij[7,15] = -0
L_ij[7,16] = -0
L_ij[

L_ij[22,42] = -0
L_ij[22,43] = -0
L_ij[22,44] = -0
L_ij[22,45] = -0
L_ij[22,46] = -0
L_ij[22,47] = -0
L_ij[22,48] = -0
L_ij[22,49] = 0
L_ij[22,50] = -0
L_ij[23,26] = 0
L_ij[23,27] = 0
L_ij[23,28] = 0
L_ij[23,29] = 0
L_ij[23,30] = -0
L_ij[23,31] = -0
L_ij[23,32] = -0
L_ij[23,33] = -0
L_ij[23,34] = -0
L_ij[23,35] = -0
L_ij[23,36] = -0
L_ij[23,37] = -0
L_ij[23,38] = -0
L_ij[23,39] = -0
L_ij[23,40] = -0
L_ij[23,41] = -0
L_ij[23,42] = -0
L_ij[23,43] = -0
L_ij[23,44] = -0
L_ij[23,45] = -0
L_ij[23,46] = -0
L_ij[23,47] = -0
L_ij[23,48] = -0
L_ij[23,49] = -0
L_ij[23,50] = -0
L_ij[24,27] = 0
L_ij[24,28] = 0
L_ij[24,29] = 0
L_ij[24,30] = 0
L_ij[24,31] = -0
L_ij[24,32] = -0
L_ij[24,33] = -0
L_ij[24,34] = -0
L_ij[24,35] = -0
L_ij[24,36] = -0
L_ij[24,37] = -0
L_ij[24,38] = -0
L_ij[24,39] = -0
L_ij[24,40] = -0
L_ij[24,41] = -0
L_ij[24,42] = -0
L_ij[24,43] = -0
L_ij[24,44] = -0
L_ij[24,45] = -0
L_ij[24,46] = -0
L_ij[24,47] = -0
L_ij[24,48] = -0
L_ij[24,49] = -0
L_ij[24,50] = -0
L_ij[25,28] = 0
L_ij[25

In [47]:
m.write("check.lp")